In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time
import os
import base64

def save_to_pdf(driver, file_path):
    parameters = {
        "printBackground": True, # 背景画像を印刷
        "paperWidth": 8.27, # A4用紙の横 210mmをインチで指定
        "paperHeight": 11.69, # A4用紙の縦 297mmをインチで指定
        # "displayHeaderFooter": True, # 印刷時のヘッダー、フッターを表示
    }
    # Chrome Devtools Protocolコマンドを実行し、取得できるBase64形式のPDFデータをデコードしてファイルに保存
    pdf_base64 = driver.execute_cdp_cmd("Page.printToPDF", parameters)
    #print(pdf_base64)
    pdf = base64.b64decode(pdf_base64["data"])
    with open(file_path, 'bw') as f:
        f.write(pdf)

# カレントディレクトリにすみぺLINEBLOGのURLリスト作成
root_url = 'https://lineblog.me/uesaka_sumire/'
urls = [root_url]

for i, n_page in enumerate(range(2,301,1),1):
    page_url = root_url + "?p={}".format(n_page)
    urls.append(page_url)
    
df = pd.DataFrame(urls)
df.to_csv('urls.txt', index=False, header=False)

# URLリスト取得
urls = []
with open('urls.txt', mode='rt', encoding='utf-8') as f:
    urls = f.readlines()

#保存先ディレクトリの作成
pdf_destination = 'Uesaka Sumire/LineBlogPDFs/'
os.makedirs(pdf_destination, exist_ok=True)

# Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')

# PDF保存実行
driver = webdriver.Chrome("./chromedriver.exe", options=options)
for i, url in enumerate(urls,1):
    driver.get(url)
    new_file_path = pdf_destination + str(i) + '.pdf'
    save_to_pdf(driver, new_file_path)
    print(new_file_path)
    # time.sleep(0.5)

driver.close()
driver.quit()